## Q1: Find how much amount spent by each customer on artists? Write a query to return customer name, artist name and total spent

In [1]:
'''Steps to Solve: First, find which artist has earned the most according to the InvoiceLines. Now use this artist to find 
which customer spent the most on this artist. For this query, you will need to use the Invoice, InvoiceLine, Track, Customer, 
Album, and Artist tables. Note, this one is tricky because the Total spent in the Invoice table might not be on a single product, 
so you need to use the InvoiceLine table to find out how many of each product was purchased, and then multiply this by the price
for each artist.'''

'Steps to Solve: First, find which artist has earned the most according to the InvoiceLines. Now use this artist to find \nwhich customer spent the most on this artist. For this query, you will need to use the Invoice, InvoiceLine, Track, Customer, \nAlbum, and Artist tables. Note, this one is tricky because the Total spent in the Invoice table might not be on a single product, \nso you need to use the InvoiceLine table to find out how many of each product was purchased, and then multiply this by the price\nfor each artist.'

In [2]:
'''
WITH best_selling_artist AS (
	SELECT artist.artist_id AS artist_id, artist.name AS artist_name, SUM(invoice_line.unit_price*invoice_line.quantity) AS total_sales
	FROM invoice_line
	JOIN track ON track.track_id = invoice_line.track_id
	JOIN album ON album.album_id = track.album_id
	JOIN artist ON artist.artist_id = album.artist_id
	GROUP BY 1
	ORDER BY 3 DESC
	LIMIT 1
)
SELECT c.customer_id, c.first_name, c.last_name, bsa.artist_name, SUM(il.unit_price*il.quantity) AS amount_spent
FROM invoice i
JOIN customer c ON c.customer_id = i.customer_id
JOIN invoice_line il ON il.invoice_id = i.invoice_id
JOIN track t ON t.track_id = il.track_id
JOIN album alb ON alb.album_id = t.album_id
JOIN best_selling_artist bsa ON bsa.artist_id = alb.artist_id
GROUP BY 1,2,3,4
ORDER BY 5 DESC;

'''

'\nWITH best_selling_artist AS (\n\tSELECT artist.artist_id AS artist_id, artist.name AS artist_name, SUM(invoice_line.unit_price*invoice_line.quantity) AS total_sales\n\tFROM invoice_line\n\tJOIN track ON track.track_id = invoice_line.track_id\n\tJOIN album ON album.album_id = track.album_id\n\tJOIN artist ON artist.artist_id = album.artist_id\n\tGROUP BY 1\n\tORDER BY 3 DESC\n\tLIMIT 1\n)\nSELECT c.customer_id, c.first_name, c.last_name, bsa.artist_name, SUM(il.unit_price*il.quantity) AS amount_spent\nFROM invoice i\nJOIN customer c ON c.customer_id = i.customer_id\nJOIN invoice_line il ON il.invoice_id = i.invoice_id\nJOIN track t ON t.track_id = il.track_id\nJOIN album alb ON alb.album_id = t.album_id\nJOIN best_selling_artist bsa ON bsa.artist_id = alb.artist_id\nGROUP BY 1,2,3,4\nORDER BY 5 DESC;\n\n'

## Q2: We want to find out the most popular music Genre for each country. We determine the most popular genre as the genre with the highest amount of purchases. Write a query that returns each country along with the top Genre. For countries where the maximum number of purchases is shared return all Genres.

In [3]:
'''Steps to Solve:  There are two parts in question- first most popular music genre and second need data at country level.'''

'Steps to Solve:  There are two parts in question- first most popular music genre and second need data at country level.'

In [4]:
'''
WITH popular_genre AS 
(
    SELECT COUNT(invoice_line.quantity) AS purchases, customer.country, genre.name, genre.genre_id, 
	ROW_NUMBER() OVER(PARTITION BY customer.country ORDER BY COUNT(invoice_line.quantity) DESC) AS RowNo 
    FROM invoice_line 
	JOIN invoice ON invoice.invoice_id = invoice_line.invoice_id
	JOIN customer ON customer.customer_id = invoice.customer_id
	JOIN track ON track.track_id = invoice_line.track_id
	JOIN genre ON genre.genre_id = track.genre_id
	GROUP BY 2,3,4
	ORDER BY 2 ASC, 1 DESC
)
SELECT * FROM popular_genre WHERE RowNo <= 1

'''

'\nWITH popular_genre AS \n(\n    SELECT COUNT(invoice_line.quantity) AS purchases, customer.country, genre.name, genre.genre_id, \n\tROW_NUMBER() OVER(PARTITION BY customer.country ORDER BY COUNT(invoice_line.quantity) DESC) AS RowNo \n    FROM invoice_line \n\tJOIN invoice ON invoice.invoice_id = invoice_line.invoice_id\n\tJOIN customer ON customer.customer_id = invoice.customer_id\n\tJOIN track ON track.track_id = invoice_line.track_id\n\tJOIN genre ON genre.genre_id = track.genre_id\n\tGROUP BY 2,3,4\n\tORDER BY 2 ASC, 1 DESC\n)\nSELECT * FROM popular_genre WHERE RowNo <= 1\n\n'

## Q3: Write a query that determines the customer that has spent the most on music for each country. Write a query that returns the country along with the top customer and how much they spent. For countries where the top amount spent is shared, provide all customers who spent this amount.

In [5]:
'''
WITH Customter_with_country AS (
		SELECT customer.customer_id,first_name,last_name,billing_country,SUM(total) AS total_spending,
	    ROW_NUMBER() OVER(PARTITION BY billing_country ORDER BY SUM(total) DESC) AS RowNo 
		FROM invoice
		JOIN customer ON customer.customer_id = invoice.customer_id
		GROUP BY 1,2,3,4
		ORDER BY 4 ASC,5 DESC)
SELECT * FROM Customter_with_country WHERE RowNo <= 1

'''

'\nWITH Customter_with_country AS (\n\t\tSELECT customer.customer_id,first_name,last_name,billing_country,SUM(total) AS total_spending,\n\t    ROW_NUMBER() OVER(PARTITION BY billing_country ORDER BY SUM(total) DESC) AS RowNo \n\t\tFROM invoice\n\t\tJOIN customer ON customer.customer_id = invoice.customer_id\n\t\tGROUP BY 1,2,3,4\n\t\tORDER BY 4 ASC,5 DESC)\nSELECT * FROM Customter_with_country WHERE RowNo <= 1\n\n'